In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community

In [30]:
%pip install --upgrade --quiet  langchain langchain-google-vertexai "langchain-google-community[featurestore]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.147.0 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from google.cloud import aiplatform
import google.cloud.bigquery as bq
import langchain
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.schema import format_document
from langchain_community.document_loaders import DirectoryLoader, TextLoader,BigQueryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore

import uuid

In [13]:
# Define our query
query = """
SELECT id,media_type,content,test_metadata 
FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
"""

# Load the data
loader = BigQueryLoader(
    query, metadata_columns=["id"], page_content_columns=["content","media_type","test_metadata"]
)
data = loader.load()

In [8]:
docs = []
docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=20, chunk_overlap  = 5)
docs = text_splitter.split_documents(docs)

In [9]:
doc_ids = [str(uuid.uuid4()) for _ in docs]
chunked_docs=[]
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    doc.metadata['chunk_id']=_id

chunked_docs.extend(docs)

In [11]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"

In [12]:
DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}


embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)


store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


In [ ]:
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

### source

https://cloud.google.com/blog/products/ai-machine-learning/rag-with-bigquery-and-langchain-in-cloud

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []
documents.extend(loader.load())
 
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
)
doc_splits = text_splitter.split_documents(documents)

# Add chunk number to metadata
chunk_idx=0
prev=doc_splits[0].metadata["id"]
for idx, split in enumerate(doc_splits):
    if prev==split.metadata["id"]:
       split.metadata["chunk"] = chunk_idx
      
    else:
        chunk_idx=0
        split.metadata["chunk"] = chunk_idx
        prev=split.metadata["id"]
    chunk_idx +=1
        

In [29]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore


embedding_model = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)
bq_store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
) 

bq_store.add_documents(doc_splits)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


['bd45145ea8f24e04969cc6eabf560871',
 '4919842ae3564ea7bdda8121e422754d',
 'f6765c7ecb5d4a739106350282e7ef17',
 'd64716721e894f08b1e665dc1295be32',
 '7633c0e021c54fbc947ca69ca7e810f2',
 '1205869d5e24424a8b5990d6f113a94c',
 '4b1a18c7f5bf49c58d2e32793af14b35',
 'f175151bb82846c789e9b73499d33a96',
 '87c0a049eaa949b5bf64fe4e2027b33f',
 'b383bee675a44725be0572ff8f75f983',
 'f471708bf0514f69bcd384f3a3f2214c',
 '0111fa8a84bc436caeaad46cd8db249f',
 'cce22929e5d4469d826ae33ab3e43f47',
 '14761c09fcbf487681d13d622d7abd0b',
 'd44db2e00c024e55a9a742d99fc0bf22',
 '2e1eb88685aa4ab9985aba69855ec2dd',
 'eb116a67477d404f89ed4c730112ca64',
 'cea83171fd24434fa978c25949bd154c',
 'ca580c13c64442bea19c036272e9b175',
 '995444bfac574b83a73e2540506687a4',
 'a68e1429c38b4002bffdbbd8da7f1da3',
 '3ef4671195594913aa17a70ac813c6ac',
 '2566970b645645a18bc5cb86481b1891',
 '25a24e94463a42ac9535a85559db14b2',
 '01f6c8ba6aaf4a71a8e1056f28468ba1',
 'aea5475aa51047ba8e43837e8f045124',
 '0b814131568f474193db1cb936b8db56',
 